In [21]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import math
import topojson as tp
import pandas as pd

In [22]:
# Load the Shapefile into a GeoDataFrame
shapefile_path = '../results/Puttaparthi_Nallamada_Gopepalli_02627900/Gopepalli_geotiff_H1/output_Gopepalli_geotiff_H1.shp'
gdf = gpd.read_file(shapefile_path)

In [23]:
def calculate_min_angle_in_degrees(polygon):
    # Extract the exterior ring of the polygon
    exterior = polygon.exterior
    # Get the coordinates of the vertices
    coords = list(exterior.coords)
    
    # Calculate the angles between consecutive edges
    angles = []
    for i in range(len(coords) - 2):
        v1 = (coords[i][0] - coords[i+1][0], coords[i][1] - coords[i+1][1])
        v2 = (coords[i+2][0] - coords[i+1][0], coords[i+2][1] - coords[i+1][1])
        dot_product = v1[0] * v2[0] + v1[1] * v2[1]
        magnitude_product = (v1[0]**2 + v1[1]**2) * (v2[0]**2 + v2[1]**2)
        angle_rad = math.acos(dot_product / math.sqrt(magnitude_product))
        angle_deg = math.degrees(angle_rad)  # Convert radians to degrees
        angles.append(angle_deg)
    
    # Find the minimum angle in degrees
    min_angle_deg = min(angles)
    
    return min_angle_deg

In [24]:
gdf['min_angle'] = gdf['geometry'].apply(calculate_min_angle_in_degrees)

In [25]:
threshold = 75
gdf_below_threshold = gdf[gdf['min_angle'] < threshold]
gdf_above_threshold = gdf[gdf['min_angle'] >= threshold]

In [26]:
# simplify polygons
topo1 = tp.Topology(gdf_above_threshold)
gdf_above_threshold = topo1.toposimplify(epsilon=1.55).to_gdf()
topo2 = tp.Topology(gdf_below_threshold)
gdf_below_threshold = topo2.toposimplify(epsilon=5, simplify_algorithm='vw', simplify_with='simplification', prevent_oversimplify=True).toposimplify(epsilon=5).to_gdf()

In [27]:
combined_gdf = pd.concat([gdf_below_threshold, gdf_above_threshold], ignore_index=True)

In [28]:
# Set your area threshold (e.g., 100 square units)
area_threshold = 10  # Adjust this value as needed

# Calculate the area of each polygon and add it as a new column
combined_gdf['area'] = combined_gdf['geometry'].area

# Filter the GeoDataFrame based on the area threshold
filtered_gdf = combined_gdf[combined_gdf['area'] >= area_threshold]

In [29]:
filtered_gdf.to_file('filtered_shapefile.shp')